# Preprocesado de Texto

In [ ]:
! pip install spacy
! python -m spacy download es
! pip install transformers
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-08 17:04:55.197337: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 21.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
def NLP_preprocesaDocumento(doc):
  #Cargamos el modelo en español
  nlp = spacy.load("es_core_news_sm")

  #Tokenizado
  doc = [" ".join([token.text for token in nlp(doc)])][0]

  #Eliminación de signos de puntuación
  doc = [" ".join([token.lemma_ for token in nlp(doc) if not token.is_punct]) ][0]

  #Eliminación de stopwords
  spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
  doc = [" ".join([token for token in doc.split() if not token.lower() in spacy_stopwords])][0]
  
  #Lematización de palabras
  doc = [" ".join([token.lemma_ for token in nlp(doc)])][0]

  #Eliminación de tildes
  tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', 'ü': 'u', 'Ü': 'U'}
  for tilde in tildes:
	  if tilde in doc:
		  doc = doc.replace(tilde, tildes[tilde])

  return doc


In [ ]:
doc = "Hay evidencia de hipoatenuación hepática difusa compatible con infiltración grasa . No hay dilatación de los conductos biliares intra o extrahepáticos. El paciente se encuentra en estado post colecistectomía. El bazo es normal. El páncreas es de contorno y características de atenuación normales. No hay evidencia de masa suprarrenal. Hay una hernia supraumbilical de tamaño moderado que contiene grasa. Los riñones son normales en tamaño, forma y configuración. No se identifican cálculos renales ni ureterales. No hay hidrouréter ni hidronefrosis. No hay evidencia de apendicitis. Hay varias asas de intestino delgado llenas de líquido, compatibles con una enteritis leve. No hay engrosamiento de la pared intestinal. No hay evidencia de obstrucción del intestino delgado o grueso. No hay evidencia de ascitis abdominal o linfadenopatía. No hay evidencia de masa vesical intrínseca o extrínseca . No hay ascitis pélvica ni linfadenopatía . El útero y los ovarios no presentan ninguna anomalía. Las imágenes de las bases pulmonares no muestran evidencia de masa pleural o parenquimatosa . No hay derrames pleurales. Hay cicatrices en el lóbulo medio derecho y en la língula, así como en ambas bases pulmonares. Las estructuras óseas están libres de lesiones líticas o blásticas. Se observan cambios degenerativos multinivel en la columna toracolumbar. Se observan calcificaciones dispersas en la aorta y en sus principales ramas, compatibles con la aterosclerosis."

In [ ]:
NLP_preprocesaDocumento(doc)

'evidenciar hipoatenuacion hepatico difusa compatible infiltracion gras dilatacion conducto biliar intra extrahepatico paciente encontrar post colecistectomia bazo normal pancrea contorno caracteristico atenuacion normal evidenciar masa suprarrenal hernia supraumbilical tamaño moderado contener grasa riñon normal tamaño forma configuracion identificar calculo renal ureteral hidroureter hidronefrosis evidenciar apendicitis asa intestino delgado llenar liquido compatible enteriti leve engrosamiento pared intestinal evidenciar obstruccion intestino delgado grueso evidencia asciti abdominal linfadenopatir evidencio masa vesical intrinseco extrinseca asciti pelvico linfadenopatir utero ovario presentar anomalio imagen base pulmonar mostrar evidencio masa pleural parenquimatos derrame pleural cicatriz lobulo derecho lingular base pulmonar estructura oseo libre lesion litico blastico observar cambio degenerativo multinivel columno toracolumbar observar calcificacion disperso aorta principal r

# Carga del modelo de Embedding BERT





Aquí es importante cargar el modelo BERT entrenado para lenguaje médico!!!

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
text = 'abdomen'

In [ ]:
#outputs = model(**inputs)
#outputs[0].shape

torch.Size([1, 3, 768])

In [ ]:
import torch

def NLP_get_BERT_embedding(model, text):
  # Add the special tokens.
  marked_text = "[CLS] " + text + " [SEP]"

  # Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)

  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  # Mark each of the tokens as belonging to sentence "1".
  segments_ids = [1] * len(tokenized_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Run the text through BERT, and collect all of the hidden states produced
  # from all 12 layers. 
  with torch.no_grad():

      outputs = model(tokens_tensor, segments_tensors)

      # Evaluating the model will return a different number of objects based on 
      # how it's  configured in the `from_pretrained` call earlier. In this case, 
      # becase we set `output_hidden_states = True`, the third item will be the 
      # hidden states from all layers. See the documentation for more details:
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2]

  # `hidden_states` has shape [13 x 1 x 22 x 768]

  # `token_vecs` is a tensor with shape [22 x 768]
  token_vecs = hidden_states[-2][0]

  # Calculate the average of all 22 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)
  return sentence_embedding

In [ ]:
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


In [ ]:
sentence_embedding

tensor([-6.0830e-01, -2.7166e-01, -3.4278e-01, -4.8407e-01, -4.7151e-02,
         1.9344e-01,  1.8054e-01,  3.8789e-01, -3.2630e-01, -9.7103e-01,
        -1.1977e-01,  1.5908e-01,  1.2633e-01, -5.9967e-01, -2.9988e-01,
         2.0238e-01, -1.0391e-01,  3.2397e-01,  6.3429e-01,  3.6771e-01,
         8.1225e-02, -1.6037e-02, -4.5312e-01, -1.8587e-01,  5.5763e-01,
        -5.7271e-02,  2.7353e-01,  5.9289e-02, -6.2920e-01,  3.2208e-01,
         8.1125e-02,  2.2946e-01,  1.4167e-01,  1.3945e-01, -4.9444e-01,
        -4.2282e-01,  1.2071e-01, -2.3081e-01, -3.4686e-01,  8.3958e-02,
        -2.4661e-02, -4.0552e-01,  8.1624e-01, -5.4481e-01,  1.5226e-01,
        -2.6216e-01, -7.5637e-01,  1.1092e-01, -3.6664e-01, -1.6258e-01,
        -8.0120e-01, -2.1154e-01, -5.0239e-01,  3.8217e-01, -1.2875e-02,
         4.4980e-02,  5.1710e-01,  1.8922e-01, -3.4892e-01, -6.6097e-01,
         7.0411e-02, -1.5129e-01,  1.4710e-01,  4.5724e-02,  8.0800e-02,
         1.3840e-01,  3.3532e-01,  6.1696e-01, -1.3

# Carga del modelo de Embedding BETO-Galén




In [ ]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.append("..")
#from nlp_utils import *

# BETO tokenizer
!pip install keras_bert
from keras_bert import load_vocabulary, Tokenizer
model_path = "drive/MyDrive/"
#https://github.com/dccuchile/beto/blob/master/config/uncased_2M/config.json
config_path = model_path + "config.json"
#https://github.com/guilopgar/ClinicalCodingTransformerES/tree/main/BETO/BETO-Galen
checkpoint_path = "drive/MyDrive/model.ckpt-1000000"
vocab_file = "vocab.txt"
tokenizer = Tokenizer(token_dict=load_vocabulary(model_path + vocab_file), pad_index=1, cased=True)

# Hyper-parameters
#text_col = "raw_text"
training = False
trainable = True
SEQ_LEN = 128
#BATCH_SIZE = 16
#EPOCHS = 28
#LR = 3e-5
#train_weight = 4
#all_abs_weight = 1

#random_seed = 0
#tf.set_random_seed(random_seed)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for keras_bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33516 sha256=d9f37f69fd3bd4be864e2001c00626f15c6e68e8ead0820dc6f34d63f6c419d3
  Stored in directory: /root/.cache/pip/wheels/a1/34/ed/6bbd71716d7bcea30d75e8bc5aeb94f4cb52636295c8343534
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12304 sha256=74f98d6bf73c8ef9ac528406ff76539d2e154c02406802251c2f9f695a9175f5
  Stored in directory: /root/.cache/pip/wheels/07/cd/a7/a8fa93f7e177eee0101fed63179f7a2fa3b53671ffaad82bfd
  Created wheel for keras-e

In [ ]:
#from keras.backend.tensorflow_backend import set_session
#from tensorflow.compat.v1.keras.backend import set_session

# Prevent GPU memory allocation problems
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#set_session(tf.Session(config=config))

In [ ]:
from keras_bert import load_trained_model_from_checkpoint

model = load_trained_model_from_checkpoint(
    config_file=config_path, 
    checkpoint_file=checkpoint_path, 
    training=training,                                       
    trainable=trainable, 
    seq_len=SEQ_LEN
)

In [ ]:
model.inputs

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Input-Token')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Input-Segment')>]


https://keras.io/api/layers/core_layers/input/

shape: A shape tuple (integers), not including the batch size. For instance, shape=(32,) indicates that the expected input will be batches of 32-dimensional vectors. Elements of this tuple can be None; 'None' elements represent dimensions where the shape is not known.

https://www.kaggle.com/code/rajmehra03/a-detailed-explanation-of-keras-embedding-layer

A Detailed Explanation of Keras Embedding Layer

In [ ]:
model.outputs

[<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'Encoder-12-FeedForward-Norm')>]

In [ ]:
e = model.eval()

AttributeError: ignored

In [ ]:
#Using BETO tokenizer, each sentence is converted into a sequence of subwords, 
#which are further converted into vocabulary indices (input IDs) and segments arrays (BERT input tensors).

tokenizer.encode('el paciente impaciente')

([4, 1039, 6420, 28601, 5], [0, 0, 0, 0, 0])

In [ ]:
tokenizer.tokenize('el paciente impaciente')

['[CLS]', 'el', 'paciente', 'impaciente', '[SEP]']

In [ ]:
text = 'el paciente impaciente'

In [ ]:
# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(text, tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    #token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids = tokenizer.encode(text)[0]
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [ ]:
input_ids = get_ids(text, tokenizer.tokenize(text), tokenizer, SEQ_LEN)
input_masks = get_masks(tokenizer.tokenize(text),SEQ_LEN)
input_segments = get_segments(tokenizer.tokenize(text),SEQ_LEN)

In [ ]:
print(input_segments)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
#pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
pool_embs, all_embs = model.predict([[tf.convert_to_tensor(input_ids),tf.convert_to_tensor(input_masks)],tf.convert_to_tensor(input_segments)])

ValueError: ignored

In [ ]:
text = 'El paciente presenta un abdomen distendido'

In [ ]:
encoded_text = tokenizer.encode(text)

In [ ]:
encoded_text

([4, 3, 30962, 6420, 4167, 1044, 24351, 9669, 30959, 1816, 5],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
encoded_text = ([1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1],[1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1])

In [ ]:
#tokens_tensor = torch.tensor(encoded_text[0])
#segments_tensor = torch.tensor(encoded_text[1])
tokens_tensor = tf.convert_to_tensor(encoded_text[0])
segments_tensor = tf.convert_to_tensor(encoded_text[1])

In [ ]:
tokens_tensor

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1],
      dtype=float32)>

In [ ]:
#outputs = model(inputs=tf.convert_to_tensor(tokenizer.encode(text)))
#outputs = model(inputs=[tokens_tensor, segments_tensor])
outputs = model(inputs=[tokens_tensor,segments_tensor])

InvalidArgumentError: ignored

In [ ]:
import torch

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
#tokenized_text = tokenizer.tokenize(marked_text)
tokenized_text = tokenizer.encode(marked_text)

# Map the token strings to their vocabulary indeces.
# indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
indexed_tokens = tokenized_text.ids
# Create inputs
input_ids = tokenized_context.ids + tokenized_question.ids[1:]
token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
    tokenized_question.ids[1:]
)

# Mark each of the tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])




AttributeError: ignored

In [ ]:
print(tokens_tensor)

tensor([[  101,  3449, 14397, 25099,  2556,  2050,  4895, 13878,  4487, 16173,
          4305,  3527,   102]])


In [ ]:
print(segments_tensors)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
inputs = [tokens_tensor, segments_tensors]

In [ ]:
print(inputs)

[tensor([[  101,  3449, 14397, 25099,  2556,  2050,  4895, 13878,  4487, 16173,
          4305,  3527,   102]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])]


In [ ]:
max_len = SEQ_LEN

input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')

attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    
embeddings = model([input_ids,attention_masks])[1]
    
output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)

In [ ]:
#outputs = model([tokens_tensor,segments_tensors])
outputs = model(inputs)


AttributeError: ignored

In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():
    print('antes de la magia')
    outputs = model(tokens_tensor, segments_tensors)
    print('después de la magia')

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
sentence_embedding = NLP_get_BERT_embedding(model, text)

# Carga de las terminologías clínicas SNOMED-CT

In [ ]:
import os
def cargaProblemasSaludAHSpainExtension():
  f = open('der2_Refset_ProblemasSaludAHSpainExtensionSnapshot_ES_20221201.txt')
  terminos_snomedct_problemassaludAH = []
  cont = 0
  for row in f:
    if cont>0 and cont<100:
      values = row.strip().split("\t")
      #id	effectiveTime	active	moduleId	refsetId	referencedComponentId	term
      #print(values)
      termino = {}
      termino["id"] = values[0];
      termino["effectiveTime"] = values[1];
      termino["active"] = values[2];
      termino["moduleId"] = values[3];
      termino["refsetId"] = values[4];
      termino["referencedComponentId"] = values[5];
      termino["term"] = values[6];
      termino["num_palabras"] = len(values[6].split(" "));
      #print(termino)
      terminos_snomedct_problemassaludAH.append(termino)
    cont = cont + 1

  f.close();
  return terminos_snomedct_problemassaludAH;

In [ ]:
def anadirEmbedding(model,lista):
  for item in lista:
    doc = NLP_preprocesaDocumento(item["term"])
    print(doc)
    item["embedding"] = NLP_get_BERT_embedding(model, doc)


In [ ]:
terminos_snomedct_problemassaludAH = cargaProblemasSaludAHSpainExtension();

In [ ]:

terminos_snomedct_problemassaludAH = anadirEmbedding(model, terminos_snomedct_problemassaludAH);

abdomar distendido
aborto espontaneo
aborto espontaneo completo
aborto espontaneo incompleto
abortir inadvertido
aborto retenido
abrasion
abrasion corneal
abrasion extremidad inferior
absceso
absceso abdominal
absceso amigdalar
absceso higado
absceso glandulo Bartolino
absceso mama
absceso piel tejido celular subcutaneo
absceso vulva
absceso pie
absceso pulmon
absceso dental
absceso periamigdalino


KeyboardInterrupt: ignored

In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  print("Término: %s, preprocesado: %s", (termino['term'], NLP_preprocesaDocumento(termino['term'])))

# Obtención de n-gramas del texto

In [ ]:
def NLP_get_ngrams(text, n):
    text = text.split()
    output = []
    for i in range(len(text) - n + 1):
        output.append(text[i:i+n])
    return output

# Probando

In [ ]:
#informe = "El paciente presenta un abdomen distendido, y no es capaz de volverse loco. El paciente presenta ansiedad y a veces un poco de apraxia."
informe = "Hay evidencia de hipoatenuación hepática difusa compatible con infiltración grasa . No hay dilatación de los conductos biliares intra o extrahepáticos. El paciente se encuentra en estado post colecistectomía. El bazo es normal. El páncreas es de contorno y características de atenuación normales. No hay evidencia de masa suprarrenal. Hay una hernia supraumbilical de tamaño moderado que contiene grasa. Los riñones son normales en tamaño, forma y configuración. No se identifican cálculos renales ni ureterales. No hay hidrouréter ni hidronefrosis. No hay evidencia de apendicitis. Hay varias asas de intestino delgado llenas de líquido, compatibles con una enteritis leve. No hay engrosamiento de la pared intestinal. No hay evidencia de obstrucción del intestino delgado o grueso. No hay evidencia de ascitis abdominal o linfadenopatía. No hay evidencia de masa vesical intrínseca o extrínseca . No hay ascitis pélvica ni linfadenopatía . El útero y los ovarios no presentan ninguna anomalía. Las imágenes de las bases pulmonares no muestran evidencia de masa pleural o parenquimatosa . No hay derrames pleurales. Hay cicatrices en el lóbulo medio derecho y en la língula, así como en ambas bases pulmonares. Las estructuras óseas están libres de lesiones líticas o blásticas. Se observan cambios degenerativos multinivel en la columna toracolumbar. Se observan calcificaciones dispersas en la aorta y en sus principales ramas, compatibles con la aterosclerosis."
informe_preprocesado = NLP_preprocesaDocumento(informe)
tokens = informe_preprocesado.split(" ");
print(tokens)

['evidenciar', 'hipoatenuacion', 'hepatico', 'difusa', 'compatible', 'infiltracion', 'gra', 'dilatacion', 'conductir', 'biliar', 'intra', 'extrahepatico', 'paciente', 'post', 'colecistectomia', 'bazo', 'normal', 'pancrea', 'contorno', 'caracteristico', 'atenuacion', 'normal', 'evidenciar', 'masa', 'suprarrenal', 'hernia', 'supraumbilical', 'tamaño', 'moderado', 'contener', 'grasa', 'riñon', 'normal', 'tamaño', 'forma', 'configuracion', 'identificar', 'calculo', 'renal', 'ureteral', 'hidroureter', 'hidronefrosis', 'evidenciar', 'apendicitis', 'asa', 'intestino', 'delgado', 'lleno', 'liquido', 'compatible', 'enteriti', 'leve', 'engrosamiento', 'pared', 'intestinal', 'evidenciar', 'obstruccion', 'intestino', 'delgado', 'grueso', 'evidenciar', 'ascitis', 'abdominal', 'linfadenopatir', 'evidenciar', 'masa', 'vesical', 'intrinseca', 'extrinseco', 'ascitis', 'pelvico', 'linfadenopatir', 'utero', 'ovario', 'presentar', 'anomalia', 'imagen', 'bas', 'pulmonar', 'mostrar', 'evidencio', 'masa', 'p

In [ ]:
import torch
#TEMPORAL, mientras no arregles de del preprocesado...
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = NLP_get_BERT_embedding(model, termino["term"])

In [ ]:
import numpy as np
from numpy.linalg import norm
from operator import itemgetter

terminos_vinculados = []

#Empezamos por ngramas más grandes a más pequeños
for num_ngrams in range(6, 0, -1):
  ngramas = NLP_get_ngrams(informe_preprocesado, num_ngrams)
  for ngrama in ngramas:
    doc = " ".join([token for token in ngrama])
    #print(doc)
    doc_embedding = NLP_get_BERT_embedding(model, doc)
    #print(doc_embedding.size())
    vector_A = doc_embedding
    #Por cada término de la terminología, comprobamos la similaridad con el ngrama previo
    terminos_candidatos = []
    for termino in terminos_snomedct_problemassaludAH:
      vector_B = termino["embedding"]

      import torch
      cos = torch.nn.CosineSimilarity(dim=0)
      cosine = cos(vector_A, vector_B)

      #cosine = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
      if cosine > 0.9:
        print(doc, " ", termino["term"], " Cosine Similarity:", cosine)
        terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "cosine": cosine})
    #Nos quedamos con el término con mayor similaridad de los obtenidos
    if terminos_candidatos:
      terminos_candidatos_sorted = sorted(terminos_candidatos, key=itemgetter('cosine'), reverse=True) 
      terminos_vinculados.append(terminos_candidatos_sorted[0])

#Eliminamos duplicados
terminos_vinculados_sorted = sorted(terminos_vinculados, key=itemgetter('term')) 
termino_ant = {"term":"", "cosine": "0.0"}
terminos_vinculados_deduplicated = []
for termino in terminos_vinculados_sorted:
  if termino["term"] != termino_ant["term"]:
    if termino_ant["term"] != "":
      terminos_vinculados_deduplicated.append(termino_ant)
    termino_ant = termino
  else:
    if termino_ant["cosine"]<termino["cosine"]:
      termino_ant = termino
terminos_vinculados_deduplicated.append(termino_ant)



evidenciar hipoatenuacion hepatico difusa compatible infiltracion   agujero retiniano redondo sin desprendimiento  Cosine Similarity: tensor(0.9011)
hepatico difusa compatible infiltracion gra dilatacion   absceso perirrectal  Cosine Similarity: tensor(0.9030)
hepatico difusa compatible infiltracion gra dilatacion   accidente durante una actividad doméstica  Cosine Similarity: tensor(0.9027)
hepatico difusa compatible infiltracion gra dilatacion   agujero retiniano redondo sin desprendimiento  Cosine Similarity: tensor(0.9055)
compatible infiltracion gra dilatacion conductir biliar   absceso perirrectal  Cosine Similarity: tensor(0.9012)
infiltracion gra dilatacion conductir biliar intra   absceso de la piel Y/O del tejido celular subcutáneo  Cosine Similarity: tensor(0.9025)
contorno caracteristico atenuacion normal evidenciar masa   accidente causado por exposición a condiciones climáticas  Cosine Similarity: tensor(0.9033)
supraumbilical tamaño moderado contener grasa riñon   absces

In [ ]:
for termino in terminos_vinculados_sorted:
  print(termino["term"], " Cosine Similarity:", termino["cosine"])

aborto retenido  Cosine Similarity: tensor(0.9230)
aborto retenido  Cosine Similarity: tensor(0.9258)
aborto retenido  Cosine Similarity: tensor(0.9039)
aborto retenido  Cosine Similarity: tensor(0.9319)
absceso de hígado  Cosine Similarity: tensor(0.9073)
absceso de la piel Y/O del tejido celular subcutáneo  Cosine Similarity: tensor(0.9025)
absceso perirrectal  Cosine Similarity: tensor(0.9012)
absceso perirrectal  Cosine Similarity: tensor(0.9097)
absceso perirrectal  Cosine Similarity: tensor(0.9103)
absceso perirrectal  Cosine Similarity: tensor(0.9008)
absceso perirrectal  Cosine Similarity: tensor(0.9067)
absceso perirrectal  Cosine Similarity: tensor(0.9015)
absceso perirrectal  Cosine Similarity: tensor(0.9182)
absceso perirrectal  Cosine Similarity: tensor(0.9143)
absceso perirrectal  Cosine Similarity: tensor(0.9055)
absceso perirrectal  Cosine Similarity: tensor(0.9122)
absceso perirrectal  Cosine Similarity: tensor(0.9122)
absceso perirrectal  Cosine Similarity: tensor(0.9

In [ ]:
for termino in terminos_vinculados_deduplicated:
  print(termino["term"], " Cosine Similarity:", termino["cosine"])

aborto retenido  Cosine Similarity: tensor(0.9319)
absceso de hígado  Cosine Similarity: tensor(0.9073)
absceso de la piel Y/O del tejido celular subcutáneo  Cosine Similarity: tensor(0.9025)
absceso perirrectal  Cosine Similarity: tensor(0.9182)
abuso de cocaína sin adicción, episódico  Cosine Similarity: tensor(0.9019)
accidente causado por exposición a condiciones climáticas  Cosine Similarity: tensor(0.9059)
accidente causado por maquinaria  Cosine Similarity: tensor(0.9247)
accidente durante una actividad doméstica  Cosine Similarity: tensor(0.9264)
accidente por cuerpo extraño - orificio  Cosine Similarity: tensor(0.9118)
accidente producido por filo de vidrio  Cosine Similarity: tensor(0.9180)
acné con comedones  Cosine Similarity: tensor(0.9163)
adenocarcinoma de intestino grueso  Cosine Similarity: tensor(0.9055)
adherencias intestinales con obstrucción  Cosine Similarity: tensor(0.9179)
afasia como efecto tardío de enfermedad cerebrovascular  Cosine Similarity: tensor(0.9071)

In [ ]:
#Nos quedamos sólo con los términos más parecidos de entre los similares (EXPERIMENTAL)

#stu_details = [["Aisha",30], ["Bhavs",40],["Cat", 35],["Sam",40],["Andre",35],["Trina",40],["Robbie",30],["Beck",35]]
all_values = [dictionary["term"] for dictionary in terminos_vinculados_deduplicated]
unique_values = set(all_values)
print(unique_values)
result = []
for value in unique_values:
  this_group = []
  for dictionary in terminos_vinculados_deduplicated:
    if dictionary["term"] == value:
      this_group.append(dictionary["term"])
  result.append(this_group)
print(result)

{'adenocarcinoma de intestino grueso', 'afasia como efecto tardío de enfermedad cerebrovascular', 'accidente por cuerpo extraño - orificio', 'adherencias intestinales con obstrucción', 'absceso perirrectal', 'accidente causado por exposición a condiciones climáticas', 'absceso de hígado', 'aborto retenido', 'absceso de la piel Y/O del tejido celular subcutáneo', 'accidente causado por maquinaria', 'agujero retiniano redondo sin desprendimiento', 'accidente durante una actividad doméstica', 'abuso de cocaína sin adicción, episódico', 'accidente producido por filo de vidrio', 'acné con comedones'}
[['adenocarcinoma de intestino grueso'], ['afasia como efecto tardío de enfermedad cerebrovascular'], ['accidente por cuerpo extraño - orificio'], ['adherencias intestinales con obstrucción'], ['absceso perirrectal'], ['accidente causado por exposición a condiciones climáticas'], ['absceso de hígado'], ['aborto retenido'], ['absceso de la piel Y/O del tejido celular subcutáneo'], ['accidente ca

In [ ]:
#Nos quedamos sólo con los términos más parecidos de entre los similares (EXPERIMENTAL - A AVANZAR)
terminos_resultado = []
for termino in terminos_vinculados_deduplicated:
  vector_A = termino["embedding"]
  termino_candidato = termino
  for termino2 in terminos_vinculados_deduplicated:
    if termino["term"] != termino2["term"]:
      vector_B = termino2["embedding"]
      
      import torch
      cos = torch.nn.CosineSimilarity(dim=0)
      cosine = cos(vector_A, vector_B)
      
      #cosine = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
      #Si los términos son similares
      if cosine > 0.70:
        #print(termino["term"], " ", termino2["term"], " Cosine Similarity:", cosine)
        if termino_candidato["cosine"]<termino2["cosine"]:
          termino_candidato=termino2
  terminos_resultado.append(termino_candidato)
  print(termino_candidato["term"], " Cosine Similarity:", termino_candidato["cosine"])
        #terminos_candidatos.append({"term": termino["term"], "embedding": termino["embedding"], "cosine": cosine})
      


aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
absceso perirrectal  Cosine Similarity: tensor(0.9182)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)
aborto retenido  Cosine Similarity: tensor(0.9319)


In [ ]:
#Eliminamos duplicados
terminos_resultado_sorted = sorted(terminos_resultado, key=itemgetter('term')) 
termino_ant = {"term":"", "cosine": "0.0"}
terminos_resultado_deduplicated = []
for termino in terminos_resultado_sorted:
  if termino["term"] != termino_ant["term"]:
    if termino_ant["term"] != "":
      terminos_resultado_deduplicated.append(termino_ant)
      print(termino_ant["term"], " Cosine Similarity:", termino_ant["cosine"])
    termino_ant = termino
  else:
    if termino_ant["cosine"]<termino["cosine"]:
      termino_ant = termino
terminos_resultado_deduplicated.append(termino_ant)
print(termino_ant["term"], " Cosine Similarity:", termino_ant["cosine"])

aborto retenido  Cosine Similarity: tensor(0.9319)
absceso perirrectal  Cosine Similarity: tensor(0.9182)


In [ ]:
for token in tokens:
  #vector_A = model.wv.get_vector(token)
  vector_A = NLP_get_BERT_embedding(model, token)
  for termino in terminos_snomedct_problemassaludAH:
    vector_B = termino["embedding"]
    import torch
    cos = torch.nn.CosineSimilarity(dim=0)
    cosine = cos(vector_A, vector_B)
    #cosine = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
    if cosine > 0.90:
      print(token, " ", termino["term"], " Cosine Similarity:", cosine)

KeyboardInterrupt: ignored